# Prompt Engineering Playbook — Colab Notebook

This notebook contains **20 prompting techniques** with runnable Python examples.
Use the first code cell to select your **model provider** (Gemini, OpenAI, Anthropic, Mistral, Cohere, etc.).

**Tips**
- For review workflows, set a lower temperature (e.g., `0.1–0.3`) for more deterministic outputs.
- Avoid adding PHI/PII unless your environment is compliant.
- For RAG and ReAct, replace the toy stubs with your real tools/APIs.


---
## 📘 Table of Techniques (Clickable & Summarized)

| # | Technique (click to jump) | Summary |
|---|----------------------------|----------|
| 1 | [Zero-shot](#1-zero-shot-prompting-no-examples-instruction-only) | No examples; direct instruction |
| 2 | [One-shot](#2-one-shot-prompting-one-example-guides-output) | One example provided to guide style |
| 3 | [Few-shot](#3-few-shot-prompting-several-examples-teach-the-pattern) | Multiple examples to infer task pattern |
| 4 | [Chain-of-Thought](#4-chain-of-thought-cot-step-by-step-reasoning) | Step-by-step reasoning to improve logic |
| 5 | [Self-Consistency](#5-self-consistency-sample-multiple-paths-and-compare) | Multiple reasoning paths → consistent result |
| 6 | [ReAct](#6-react-reason--act--decide-call-a-tool-then-answer) | Combine reasoning + tool use |
| 7 | [Tree-of-Thought](#7-tree-of-thought--branch-and-evaluate-reasoning-paths) | Explore reasoning branches; pick best |
| 8 | [Generated Knowledge](#8-generated-knowledge--create-background-then-answer) | Generate context first, then solve |
| 9 | [RAG](#9-retrieval-augmented-generation-rag--toy-retrieval-stub) | Retrieve external docs before answering |
| 10 | [Instruction](#10-instruction-prompting--explicit-structured-directive) | Direct, rule-based task prompt |
| 11 | [Contextual](#11-contextual-prompting--add-situational-background) | Add domain or role context |
| 12 | [Role](#12-role-prompting--assign-a-persona-for-toneauthority) | Act as a domain expert |
| 13 | [CoT Explicit Steps](#13-cot-with-explicit-steps--numbered-phases) | Structured numbered reasoning |
| 14 | [Multi-turn](#14-multi-turn-prompting--keep-brief-history-across-turns) | Multi-round conversation w/ memory |
| 15 | [Program-Aided](#15-program-aided-prompting-pap--combine-code--llm) | Combine computation + language model |
| 16 | [Least-to-Most](#16-least-to-most--solve-easy-subproblems-first) | Break down complex → simple parts |
| 17 | [Meta](#17-meta-prompting--prompts-about-prompts) | Improve prompts through reflection |
| 18 | [Automatic Prompt Engineering](#18-automatic-prompt-engineering-ape--toy-candidate-scoring) | Auto-generate optimized prompts |
| 19 | [Multimodal](#19-multimodal-prompting--placeholder-for-text--image-models) | Combine text + images |
| 20 | [Contrastive](#20-contrastive-prompting--compare-alternatives-explicitly) | Compare perspectives for balance |


In [ ]:

# =========================[ Provider Setup ]=========================
# Choose ONE provider below (uncomment its block) and set your API key.
# The helper `llm(prompt, model=..., temperature=...)` keeps the rest
# of the notebook vendor-agnostic.
#
# If nothing is configured, calling `llm(...)` will raise a helpful error.
# ===================================================================

TEMPERATURE = 0.2

# 1) Google Gemini (google-generativeai)
# !pip -q install google-generativeai
# import google.generativeai as genai
# import os
# genai.configure(api_key=os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY"))
# def llm(prompt, model="gemini-1.5-pro", temperature=TEMPERATURE):
#     return genai.GenerativeModel(model).generate_content(
#         prompt,
#         generation_config={"temperature": temperature}
#     ).text

# 2) OpenAI
# !pip -q install openai
# from openai import OpenAI
# import os
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# def llm(prompt, model="gpt-4.1-mini", temperature=TEMPERATURE):
#     r = client.chat.completions.create(
#         model=model,
#         temperature=temperature,
#         messages=[{"role":"user","content":prompt}]
#     )
#     return r.choices[0].message.content

# 3) Anthropic Claude
# !pip -q install anthropic
# import anthropic, os
# client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
# def llm(prompt, model="claude-3-5-sonnet-20241022", temperature=TEMPERATURE):
#     msg = client.messages.create(
#         model=model,
#         temperature=temperature,
#         max_tokens=1200,
#         messages=[{"role":"user","content":prompt}]
#     )
#     # Concatenate text blocks
#     out = []
#     for blk in msg.content:
#         if getattr(blk, "type", "") == "text":
#             out.append(blk.text)
#     return "\n".join(out) if out else str(msg)

# 4) Mistral
# !pip -q install mistralai
# from mistralai.client import MistralClient
# import os
# client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"))
# def llm(prompt, model="mistral-large-latest", temperature=TEMPERATURE):
#     r = client.chat(model=model, temperature=temperature, messages=[{"role":"user","content":prompt}])
#     return r.choices[0].message.content

# 5) Cohere
# !pip -q install cohere
# import cohere, os
# co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))
# def llm(prompt, model="command-r-plus", temperature=TEMPERATURE):
#     r = co.chat(model=model, temperature=temperature, messages=[{"role":"user","content":prompt}])
#     return r.text

def not_configured():
    raise RuntimeError("Please configure ONE provider in the setup cell above (uncomment a block and add your API key/env var).")

try:
    llm  # type: ignore
except NameError:
    def llm(prompt, model=None, temperature=TEMPERATURE):
        return not_configured()


> **Batching:** For bulk reviews, wrap calls in loops and store outputs (CSV/DB).
"
"> **Guardrails:** When requiring JSON, validate with a schema and retry on failure.
"
"> **RAG:** Replace toy retrieval with FAISS/Chroma/pgvector, and add citations in outputs."


---

## 1) Zero-shot prompting — No examples; instruction only.

In [ ]:
prompt = """Summarize this quarterly financial risk report into 5 bullets, plain English:
[PASTE REPORT TEXT HERE]"""
print(llm(prompt))

---

## 2) One-shot prompting — One example guides output.

In [ ]:
example = """Example summary style:
- Key risk drivers: liquidity, FX exposure
- Material changes: inventory +12% QoQ
- Action items: hedge EUR, tighten DSO
"""
prompt = example + "\nNow summarize the new report similarly:\n[NEW REPORT]"
print(llm(prompt))

---

## 3) Few-shot prompting — Several examples teach the pattern.

In [ ]:
shots = """
Task: Classify transaction as FRAUD or OK.

Example 1:
Input: Merchant=ABC Travel, Amount=$4,920, Country=US
Label: OK

Example 2:
Input: Merchant=CryptoX, Amount=$9,990, Country=RU
Label: FRAUD
"""
prompt = shots + "\nPredict label for:\nInput: Merchant=XYZ GiftCards, Amount=$7,500, Country=Unknown\nLabel:"
print(llm(prompt))

---

## 4) Chain-of-Thought (CoT) — Step-by-step reasoning.

In [ ]:
prompt = """Analyze these ratios and conclude company health.
Think step-by-step (show brief reasoning, then final verdict):

Data:
- Current ratio 1.9
- Debt/Equity 0.6
- Gross margin 42%

Output: Reasoning (brief) -> Verdict
"""
print(llm(prompt))

---

## 5) Self-consistency — Sample multiple paths and compare.

In [ ]:
import collections
prompt = "Given symptoms: fever, rash, joint pain—list top likely diagnoses (3)."
votes = collections.Counter(llm(prompt) for _ in range(3))
print("Consensus:", votes.most_common(1)[0][0])
print("\nAll candidates:\n", "\n---\n".join(votes.keys()))

---

## 6) ReAct (Reason + Act) — Decide, call a tool, then answer.

In [ ]:
def search_db(query:str)->str:
    # Replace with your real DB/search function
    return "DB_RESULT: Latest EPS $1.22, YoY +8%."

plan = llm("""You can REASON then ACT with tools.
Question: "Is the company's EPS trending up?"
Think: We should fetch EPS.
Act: search_db("EPS trend for ACME")""")

tool_result = search_db("EPS trend for ACME")
final = llm(f"Context from tool: {tool_result}\nAnswer the original question succinctly.")

print("Plan:\n", plan, "\n")
print("Tool result:\n", tool_result, "\n")
print("Final:\n", final)

---

## 7) Tree-of-Thought — Branch and evaluate reasoning paths.

In [ ]:
question = "Choose best portfolio: {A: 70/30}, {B: 50/50}, {C: 30/70} for a 5-yr horizon."
branches = [llm(f"Option {opt}: pros/cons, risk, return, suitability for moderate-risk investor.") for opt in ["A","B","C"]]
judge = llm("Evaluate the following analyses and pick the best for a moderate-risk investor:\n" + "\n\n".join(branches))
print("=== Branches ===\n" + "\n\n".join(branches))
print("\n=== Verdict ===\n" + judge)

---

## 8) Generated Knowledge — Create background, then answer.

In [ ]:
knowledge = llm("Summarize company ACME: products, markets, risks, last 2 years trends.")
answer = llm(f"Using this background:\n{knowledge}\nNow: Assess ACME’s liquidity risks in 5 bullets.")
print("Background:\n", knowledge, "\n")
print("Assessment:\n", answer)

---

## 9) Retrieval-Augmented Generation (RAG) — Toy retrieval stub.

In [ ]:
retrieved = [
    "10-K excerpt: Operating cash flow up 12%, CAPEX stable.",
    "Earnings call: Guidance raised for FY, FX headwinds easing."
]
prompt = f"Ground your answer ONLY in these docs:\n{retrieved[0]}\n{retrieved[1]}\n\nQuestion: Summarize growth drivers."
print(llm(prompt))

---

## 10) Instruction prompting — Explicit, structured directive.

In [ ]:
prompt = """Extract all dollar amounts > $10,000 and return JSON array of numbers only.
Text:
- Purchase: $4,500
- Equipment: $25,000
- Settlement: $180,000
- Fees: $9,900
"""
print(llm(prompt))

---

## 11) Contextual prompting — Add situational background.

In [ ]:
prompt = """Context: You are a senior medical reviewer.
Task: Turn these notes into a concise assessment & plan (<=120 words).
Notes: [PASTE CLINICAL NOTES]
"""
print(llm(prompt))

---

## 12) Role prompting — Assign a persona for tone/authority.

In [ ]:
prompt = """You are a compliance officer. Identify high-risk clauses in this NDA and explain why in 3 bullets:
[PASTE NDA]
"""
print(llm(prompt))

---

## 13) CoT with Explicit Steps — Numbered phases.

In [ ]:
prompt = """Follow these steps:
1) Read contract text
2) Extract obligations per party
3) List 3 risks with brief rationale
Contract:
[PASTE CONTRACT]
Output format:
- Obligations: {Party A:[], Party B:[]}
- Risks: [..]
"""
print(llm(prompt))

---

## 14) Multi-turn prompting — Keep brief history across turns.

In [ ]:
history = []
def chat(user):
    history.append({"role":"user","content":user})
    response = llm("Conversation:\n"+str(history)+"\nAssistant:")
    history.append({"role":"assistant","content":response})
    return response

print(chat("We saw revenue up 12%. Do we have regional breakdown?"))
print(chat("Yes, EMEA led. Summarize next steps."))

---

## 15) Program-Aided Prompting (PAP) — Combine code + LLM.

In [ ]:
ratios = {"current":1.9, "quick":1.5, "de_ratio":0.6}
prompt = f"Given ratios = {ratios}, provide a concise health assessment (<=80 words)."
print(llm(prompt))

---

## 16) Least-to-Most — Solve easy subproblems first.

In [ ]:
easy = llm("Compute revenue growth: 120M -> 138M YoY. Give % growth only.")
hard = llm(f"Given growth={easy}, discuss sustainability in 4 bullets (drivers, risks, outlook, watchlist).")
print("Easy step:\n", easy, "\n")
print("Hard step:\n", hard)

---

## 17) Meta prompting — Prompts about prompts.

In [ ]:
prompt = """I need a better prompt to extract ‘change of control’ clauses reliably.
Suggest 3 improved prompts with rationale, and one evaluation metric to compare them.
"""
print(llm(prompt))

---

## 18) Automatic Prompt Engineering (APE) — Toy candidate scoring.

In [ ]:
candidates = [
    "Extract change-of-control clauses. Return standardized JSON with clause text and risk level.",
    "Identify change-of-control clauses. Add span indices and a one-line rationale.",
    "Find ALL change-of-control clauses; output CSV fields: start,end,text,risk(score 1-5)."
]
scores = []
for p in candidates:
    grade = llm(f"Rate this prompt for recall & precision (0-10 each), then give total:\n{p}")
    scores.append((p, grade))
print("=== Candidates & Grades ===")
for i,(p,g) in enumerate(scores, 1):
    print(f"{i}) {p}\nGrade:\n{g}\n")

---

## 19) Multimodal prompting — Placeholder for text + image models.

In [ ]:
# In a real environment, pass text + image to a multimodal model (e.g., Gemini, GPT-4o, Claude Vision).
# Example (pseudo):
# from PIL import Image
# img = Image.open("/content/xray.png")
# prompt = "Given this chest X-ray, list 3 notable findings with caveats."
# response = multimodal_model.generate([prompt, img])
print("[Multimodal placeholder] Provide both text + image to a multimodal model's SDK.")

---

## 20) Contrastive prompting — Compare alternatives explicitly.

In [ ]:
prompt = """Compare two treatments (A: ACE inhibitor, B: ARB) for hypertension.
Provide: efficacy, side effects, cost, guideline stance. End with a balanced recommendation."""
print(llm(prompt))